In [1]:
# Import necessary modules
import torch
import torch.nn.functional as F

# Define the model
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = torch.nn.Linear(5, 3)  # first linear transformation
        self.fc2 = torch.nn.Linear(3, 1)  # second linear transformation

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)  # first activation layer
        x = self.fc2(x)
        output = torch.sigmoid(x)  # second activation layer (sigmoid to output probabilities)
        return output


As I progress through Part 2 of the fastai course, I've discovered the value of occasionally retracing my steps to assess my grasp on the fundamental concepts. In my journey through Part 1, I recall Chapter 4 of the book being particularly daunting, as it covered a broad range of essential topics early on. However, this comprehensive approach was necessary to establish a strong foundation for everything I would subsequently learn.

Whenever I revisit those earlier chapters and analyze the code line by line, I'm often pleasantly surprised by the level of intuition I've developed in certain areas. Simultaneously, I'm also confronted with aspects that still elude my immediate understanding. Engaging in quick drills to reinforce these concepts has proven to be an effective method for staying sharp and achieving complete comprehension, even if it may feel somewhat repetitive. It's like eating vegetables as a child – you may not want to, but it's ultimately beneficial and necessary for growth. I believe that through a similar positive feedback loop, I will acquire a taste for this process and derive great rewards from it.

So, grab a fork and let's dig in

## Predicting Bank Loan Default from Synthetic Data

For this exercise I'm going to create synthetic data to predict whether a bank loan will default. I've chosen the following to create as features:

- Loan Amount 
- Term 
- Interest Rate 
- Borrower's Income 
- Borrower's Credit Score

## Generate Synthetic Data

In [2]:
torch.set_printoptions(precision=4, linewidth=140, threshold=500, sci_mode=False)

We're first going to create the data for 1000 samples randomly with `torch.normal`, to which we'll pass arguments for a *mean*, *standard deviation*, and *size*.

The size could be either a tuple or a list, but for this exercise we'll just use a tuple with a single value.

Also worth noting that `term` will be using `randint` rather than `normal`, because loan terms are generally in whole months.

In [3]:
n_samples = 1000
loan_amount = torch.normal(5000., 1500, size=(n_samples,))  # average loan amount is $5000
term = torch.randint(12, 60, size=(n_samples,))  # loan term varies between 1 and 5 years
interest_rate = torch.normal(0.05, 0.01, size=(n_samples,))  # average interest rate is 5%
income = torch.normal(50000, 10000, size=(n_samples,))  # average income is $50,000
credit_score = torch.normal(600, 50, size=(n_samples,))  # average credit score is 600

Next we'll stack the tensors and create a target variable where 10% of loans default, and convert them to *float32* data types in the process

In [4]:
x = torch.column_stack([loan_amount, term, interest_rate, income, credit_score]).float()
y = torch.distributions.categorical.Categorical(torch.tensor([0.9, 0.1])).sample((n_samples,)).float()

We'll also need to add a unit axis to our dependent variable so that it can be properly broadcast.

In [5]:
y.shape

torch.Size([1000])

This can be done using either `view` or specifying `None` on the axis we want to create

In [6]:
y.view(-1,1).shape

torch.Size([1000, 1])

In [7]:
y[:, None].shape

torch.Size([1000, 1])

In [8]:
y = y[:, None]

In [9]:
y.shape

torch.Size([1000, 1])

Now that we have our data, let's move onto the creation of our neural network!

## Creating a Simple Neural Network with Pytorch

For this exercise, we're going to use 2 linear transformation layers, and a 3 activation layers

In [10]:
lin_1 = torch.nn.Linear(5, 50)
lin_2 = torch.nn.Linear(50, 1)
relu = torch.nn.functional.relu
sigmoid = torch.nn.functional.sigmoid

In [11]:
step_1 = lin_1(x)
step_1.shape

torch.Size([1000, 50])

In [12]:
step_2 = relu(step_1)
step_2, step_2.shape

(tensor([[16407.9531,     0.0000,     0.0000,  ...,     0.0000,     0.0000,     0.0000],
         [18203.2441,     0.0000,     0.0000,  ...,     0.0000,     0.0000,     0.0000],
         [17302.1680,     0.0000,     0.0000,  ...,     0.0000,     0.0000,     0.0000],
         ...,
         [17365.6875,     0.0000,     0.0000,  ...,     0.0000,     0.0000,     0.0000],
         [15493.4541,     0.0000,     0.0000,  ...,     0.0000,     0.0000,     0.0000],
         [17931.8848,     0.0000,     0.0000,  ...,     0.0000,     0.0000,     0.0000]], grad_fn=<ReluBackward0>),
 torch.Size([1000, 50]))

In [13]:
step_3 = lin_2(step_2)
step_3

tensor([[-2019.1620],
        [-1994.7572],
        [-1871.0475],
        ...,
        [-1916.7795],
        [-1722.8459],
        [-1979.5967]], grad_fn=<AddmmBackward0>)